In [24]:
!pip install PyPDF2 requests

In [25]:
import math
import requests
from PyPDF2 import PdfReader, PdfWriter

birth_date = 24
birth_month = 3
birth_year = 2001

if birth_month <= 7:
    book_number = birth_month
else:
    book_number = math.ceil(birth_month / 2)

print(f"Selected Book Number: {book_number}")

pdf_path = "/content/harrypotter.pdf"
reader = PdfReader(pdf_path)
total_pages = len(reader.pages)

print(f"Total pages in combined PDF: {total_pages}")

# ---------------- BOOK TITLES ----------------
book_titles = [
    "Harry Potter and the Sorcerer's Stone",
    "Harry Potter and the Chamber of Secrets",
    "Harry Potter and the Prisoner of Azkaban",
    "Harry Potter and the Goblet of Fire",
    "Harry Potter and the Order of the Phoenix",
    "Harry Potter and the Half-Blood Prince",
    "Harry Potter and the Deathly Hallows"
]

# book title according to birth month
title = book_titles[book_number - 1]
print(f"Selected Book Title: {title}")

Selected Book Number: 3
Total pages in combined PDF: 3623
Selected Book Title: Harry Potter and the Prisoner of Azkaban


In [ ]:
reader = PdfReader(pdf_path)
writer = PdfWriter()

total_pages = len(reader.pages)
print(f"Total pages in PDF: {total_pages}")

# ---------------- GIVEN PAGE RANGE ----------------
start_page = 568
end_page = 940

# Convert to 0-based indexing
start_index = start_page - 1
end_index = end_page - 1

print(f"Extracting pages {start_page} to {end_page}...")

# ---------------- EXTRACT PAGES ----------------
for page_num in range(start_index, end_index + 1):
    writer.add_page(reader.pages[page_num])

# ---------------- SAVE NEW PDF ----------------
output_file = "Prisoner_of_Azkaban.pdf"

with open(output_file, "wb") as f:
    writer.write(f)

print(f"\nBook extracted successfully as: {output_file}")

Total pages in PDF: 3623
Extracting pages 568 to 940...

Book extracted successfully as: Prisoner_of_Azkaban.pdf


In [ ]:
book_pdf = "/content/Prisoner_of_Azkaban.pdf"
reader = PdfReader(book_pdf)
total_pages = len(reader.pages)

print(f"Total pages in selected book: {total_pages}")

start_page1 = birth_date
end_page1 = start_page1 + 9   # next 10 pages

# Convert to 0-based indexing
start_index1 = start_page1 - 1
end_index1 = end_page1 - 1

print(f"Extracting file1.txt from pages {start_page1} to {end_page1}")

text1 = ""

for page_num in range(start_index1, min(end_index1 + 1, total_pages)):
    text1 += reader.pages[page_num].extract_text()

with open("file1.txt", "w", encoding="utf-8") as f:
    f.write(text1)

print("file1.txt created successfully.")

Total pages in selected book: 373
Extracting file1.txt from pages 24 to 33
file1.txt created successfully.


In [ ]:
year_last2 = birth_year % 100

if birth_year >= 2000:
    page_number = int("1" + f"{year_last2:02d}")
else:
    page_number = year_last2

start_page2 = page_number
end_page2 = start_page2 + 9

start_index2 = start_page2 - 1
end_index2 = end_page2 - 1

print(f"Extracting file2.txt from pages {start_page2} to {end_page2}")
text2 = ""

for page_num in range(start_index2, min(end_index2 + 1, total_pages)):
    text2 += reader.pages[page_num].extract_text()

with open("file2.txt", "w", encoding="utf-8") as f:
    f.write(text2)

print("file2.txt created successfully.")

Extracting file2.txt from pages 101 to 110
file2.txt created successfully.


Question 1

WORD COUNT (file1.txt)

In [ ]:
!pip install mrjob pyspellchecker

In [ ]:
%%writefile wordcount_mrjob.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import string

class WordCount(MRJob):
    def steps(self):
        return [
            MRStep(mapper=self.map_function,
                   reducer=self.reduce_function)
        ]
    def map_function(self, _, line):
        line = line.translate(str.maketrans('', '', string.punctuation))
        words = line.lower().split()

        for word in words:
            if word.isalpha():
                yield word, 1

    def reduce_function(self, word, counts):
        yield word, sum(counts)

if __name__ == '__main__':
    WordCount.run()

Overwriting wordcount_mrjob.py


In [ ]:
!python wordcount_mrjob.py file1.txt > wordcount_output.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/wordcount_mrjob.root.20260226.205258.320830
Running step 1 of 1...
job output is in /tmp/wordcount_mrjob.root.20260226.205258.320830/output
Streaming final output from /tmp/wordcount_mrjob.root.20260226.205258.320830/output...
Removing temp directory /tmp/wordcount_mrjob.root.20260226.205258.320830...


Question 2

NON-ENGLISH WORDS (file2.txt)

In [ ]:
%%writefile nonenglish_mrjob.py
from mrjob.job import MRJob
from mrjob.step import MRStep
from spellchecker import SpellChecker
import string


class NonEnglish(MRJob):
    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_init,
                   mapper=self.map_function,
                   reducer=self.reduce_function)
        ]
    def mapper_init(self):
        self.spell = SpellChecker()

    def map_function(self, _, line):
        line = line.translate(str.maketrans('', '', string.punctuation))
        words = line.lower().split()

        for word in words:
            if word.isalpha() and word not in self.spell:
                yield word, 1

    def reduce_function(self, word, counts):
        yield word, sum(counts)

if __name__ == '__main__':
    NonEnglish.run()

Overwriting nonenglish_mrjob.py


In [ ]:
!python nonenglish_mrjob.py file2.txt > nonenglish_output.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/nonenglish_mrjob.root.20260226.210107.635989
Running step 1 of 1...
job output is in /tmp/nonenglish_mrjob.root.20260226.210107.635989/output
Streaming final output from /tmp/nonenglish_mrjob.root.20260226.210107.635989/output...
Removing temp directory /tmp/nonenglish_mrjob.root.20260226.210107.635989...
